In [10]:
import pandas as pd
import lxml.etree as etree
import os

# Define your file path - ensure the .kml is in the same folder as your notebook
kml_file = '1DrainageRegions_SouthAfrica__doc.kml'
output_csv = 'cleaned_drainage_regions.csv'

def extract_kml_to_df(path):
    # Parse KML with lxml
    with open(path, 'rb') as f:
        tree = etree.parse(f)
    
    root = tree.getroot()
    # KML 2.0 namespace used in DWS files
    ns = {'kml': 'http://earth.google.com/kml/2.0'}
    
    data = []
    
    # Iterate through all Placemarks (these are your 'Region' entries)
    for placemark in root.xpath('//kml:Placemark', namespaces=ns):
        # Extract the text labels
        name = placemark.xpath('kml:name/text()', namespaces=ns)
        description = placemark.xpath('kml:description/text()', namespaces=ns)
        
        # Clean the text (remove newlines and extra spaces)
        name_clean = name[0].strip() if name else "Unknown"
        desc_clean = description[0].strip() if description else "No Description"
        
        data.append({
            'Region_Name': name_clean,
            'Region_Description': desc_clean
        })
    
    return pd.DataFrame(data)

# Execute extraction
df_regions = extract_kml_to_df(kml_file)

# Save to CSV
df_regions.to_csv(output_csv, index=False)

print(f"File cleaned! Saved {len(df_regions)} regions to {output_csv}")
df_regions.head()

FileNotFoundError: [Errno 2] No such file or directory: '1DrainageRegions_SouthAfrica__doc.kml'

In [6]:
valid_layers = [l for l in layers if "ORDER" in l or "River" in l]

In [8]:
import geopandas as gpd
import pandas as pd

gdfs = []

for layer in valid_layers:
    g = gpd.read_file(kml_path, layer=layer)
    if not g.empty:
        g["source_layer"] = layer
        gdfs.append(g)

gdf = gpd.GeoDataFrame(
    pd.concat(gdfs, ignore_index=True),
    crs="EPSG:4326"
)

gdf.shape

ValueError: No objects to concatenate